### Grid read & to point
* CDO를 이용하여 /DAIO/nwp/stn/{stn_id}" 경로에 지점 추출자료 생성(nc포맷)

In [1]:
import numpy as np
import pandas as pd 

# set para
stn_latlon = {"47105": [37.7515, 128.891]}

print(stn_latlon.items())
print(len(stn_latlon))

dict_items([('47105', [37.7515, 128.891])])
1


In [4]:
#!pip install cdo
from cdo import Cdo
#import cdo
import glob
import sys
import os

#
cdo = Cdo()


# Get all files in the current directory
#input_dir = "/Users/mankikim/JOB/DATA/"    # for mac
input_dir = "/Volumes/Rev_SSD/NWP/"
#input_dir = "D:/KMK_DATA/NWP/"    # for tjpc

files = glob.glob(input_dir + "*.nc")

# nearest grid 추출
for i, file in enumerate(files):
    date = file.split(".")[1]
    for key, value in stn_latlon.items():
        print(i, key, value, file)
        stn_id = key
        lat = value[0]
        lon = value[1]

        cdo_parameters = f'lon={lon}_lat={lat}'

        if not os.path.exists(f"../DAIO/nwp/stn/{key}"): os.makedirs(f"../DAIO/nwp/stn/{key}")

        output_name = f"../DAIO/nwp/stn/{key}/ldaps_stn_{key}_{date}.nc"

        # 파일의 존재 여부를 확인합니다.
        if os.path.exists(output_name):
            print(f"파일이 존재합니다. continue {output_name}")
            continue
        else:
            #print("파일이 존재하지 않습니다.")
            cdo.remapnn(cdo_parameters, input=file, output=output_name)

47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]
47105 [37.7515, 128.891]


KeyboardInterrupt: 

### Extract Lat, Lon

In [32]:
import numpy as np
import netCDF4 as nc

# Get all files in the current directory
input_dir = "/Users/mankikim/JOB/DATA/"
files = glob.glob(input_dir + "*.nc")
files[0]

# nearest grid 추출
latlon_file = nc.Dataset(files[0])
nc_lat = np.array(latlon_file.variables['latitude'])
nc_lon = np.array(latlon_file.variables['longitude'])
with open('../DAIO/nwp/ldaps_lat_grid', 'wb') as f:
    np.save(f, nc_lat)
with open('../DAIO/nwp/ldaps_lon_grid', 'wb') as f:
    np.save(f, nc_lon)

### Make [batch, sequence, features] using point nc

In [6]:
import numpy as np
import netCDF4 as nc

# .. 날짜 초기화
set_dates = pd.date_range('2021-01-14 09:00:00', '2023-05-31 09:00:00', freq='D')
set_dates = set_dates.strftime("%Y%m%d00")
#print("data day수: ", len(data_dates))
print("있어야할 day수: ", len(set_dates))

# 추출 변수 셋, 변수 설정 다하기 
col_list= ['UGRD_10maboveground',
           'VGRD_10maboveground',
           'MAXGUST_0maboveground',
           'TMP_1_5maboveground',
           'NCPCP_surface', 
           'PRES_surface']

# .. nwp 변수 초기화
nwp_data = np.full( (len(set_dates),49,len(col_list)), np.nan )

# 있는 파일만 읽어 인덱스로 입력하기, nan으로 배열이 초기화 되어 있으니까 날짜에 대한 파일을 전부다 조회할 필요 x
for key, value in stn_latlon.items(): 

   # 지점 추출 nwp file
   input_dir = f"../DAIO/nwp/stn/{key}/"
   files = glob.glob(input_dir + "*.nc")
   #print(files)

   for i,file in enumerate(files):
         
         date = file.split("_")[3].split(".")[0]
         #print("date", date)
         index = set_dates.get_loc(date)
         
         # file set
         try:
            ncfile = nc.Dataset(file)
            #ncfile
            print("Read: ", file)

            #if i == 0:
            #    nc_lat = ncfile.variables['lat']
            #    nc_lon = ncfile.variables['lon']

            for j, var in enumerate(col_list):
               a = np.array(ncfile.variables[var]).flatten()
               #nwp_data[index,:,j] = a[:-1] # 0 ~ +47
               nwp_data[index,:,j] = a[:] # 0 ~ +48

         except Exception as e:
            print(e)

있어야할 day수:  868
Read:  ../DAIO/nwp/stn/47105/ldaps_stn_47105_2023040300.nc
Read:  ../DAIO/nwp/stn/47105/ldaps_stn_47105_2023041100.nc
Read:  ../DAIO/nwp/stn/47105/ldaps_stn_47105_2023040600.nc
Read:  ../DAIO/nwp/stn/47105/ldaps_stn_47105_2023041000.nc


In [10]:
ncfile.variables['lon']

<class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (1,)
filling off

In [104]:
index = set(np.where(np.isnan(nwp_data))[0])
print(sorted(index))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [105]:
with open('../DAIO/nwp_data', 'wb') as f:
    np.save(f, nwp_data)

In [84]:
# .. nco read
#from nco import Nco
#import nco
#a = nco.Nco()
#nco = Nco()
#temperatures = nco.ncra(input=file, returnArray=True).variables['NCPCP_surface'][:]


# .. nc read
col_list= ['UGRD_10maboveground',
           'VGRD_10maboveground',
           'MAXGUST_0maboveground',
           'TMP_1_5maboveground',
           'NCPCP_surface', 
           'PRES_surface']
print(ncfile.variables.keys())
print(a[:-1].shape)

#ncfile.variables

dict_keys(['time', 'lon', 'lat', 'NDNSW_surface', 'NCPCP_surface', 'SNOL_surface', 'UGRD_10maboveground', 'VGRD_10maboveground', 'TMP_1_5maboveground', 'TMIN_1_5maboveground', 'TMAX_1_5maboveground', 'SPFH_1_5maboveground', 'RH_1_5maboveground', 'VIS_1_5maboveground', 'DPT_1_5maboveground', 'MAXGUST_0maboveground', 'LCDC_entireatmosphere_consideredasasinglelayer_', 'MCDC_entireatmosphere_consideredasasinglelayer_', 'HCDC_entireatmosphere_consideredasasinglelayer_', 'TCAR_entireatmosphere_consideredasasinglelayer_', 'PRMSL_meansealevel', 'TMP_surface', 'PRES_surface'])
(48,)
